# Character Recurrent Neural Network
- mimicing Shakespeare's writing style
- LSTM

In [1]:
"""
!rm -r data
import os 

try:
  os.mkdir("./data")
except:
  pass

!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt -P ./data
"""
print()

## 1. Settings
### 1) Import required libraries

In [2]:
import torch
import torch.nn as nn

In [3]:
import unidecode
import string
import random
import re
import time, math

## 2) Hyperparameter

In [4]:
num_epochs = 2000
print_every = 100
plot_every = 10
chunk_len = 200
hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

## 2. Data
### 1) Prepare characters

In [5]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


### 2) Get text data

In [6]:
file = unidecode.unidecode(open('./data/input.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394


## 3. Functions for text processing
### 1) Random Chunk

In [7]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

 opinion,
Make yourselves scabs?

First Citizen:
We have ever your good word.

MARCIUS:
He that will give good words to thee will flatter
Beneath abhorring. What would you have, you curs,
That like nor


### 2) Character to tensor

In [8]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


### 3) Chunk into input & label

In [9]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

## 3. Model & Optimizer
### 1) Model

In [10]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.rnn = nn.LSTM(self.embedding_size,self.hidden_size,self.num_layers)
        self.decoder = nn.Linear(self.hidden_size, self.output_size)
        
    
    def forward(self, input, hidden, cell):
        out = self.encoder(input.view(1,-1))
        out,(hidden,cell) = self.rnn(out,(hidden,cell))
        out = self.decoder(out.view(batch_size,-1))
        return out,hidden,cell

    def init_hidden(self):
        hidden = torch.zeros(self.num_layers,batch_size,self.hidden_size)
        cell = torch.zeros(self.num_layers,batch_size,self.hidden_size)
        return hidden,cell
    

model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)

In [11]:
inp = char_tensor("A")
print(inp)
hidden,cell = model.init_hidden()
print(hidden.size())

out,hidden,cell = model(inp,hidden,cell)
print(out.size())

tensor([36])
torch.Size([1, 1, 100])
torch.Size([1, 100])


### 2) Loss & Optimizer

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

### 3) Test function

In [13]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden,cell = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden,cell = model(x,hidden,cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

## 4. Train

In [14]:
for i in range(num_epochs):
    inp,label = random_training_set()
    hidden,cell = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden,cell = model(x,hidden,cell)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 200 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n\n")


 tensor([4.5826], grad_fn=<DivBackward0>) 

bS1K`IJla"`Vg%d
e@3k<r;K[7qjNJOl=I[ELg:rwX6a|^@E-G

d6y[v1<pfL<d:|13]u~)sKpn/]NK>!+Z^ZS,
jU.q#a'V7_2=J9v1@Hq;l;G;_/Y&



 tensor([2.3578], grad_fn=<DivBackward0>) 

bwnrans venis here, the thit bpane I fat, uir ang in to and,
Thane the and wat meche pe? ome ord da enof tours the ind fom of romat and thatet fouaf it is, hter nof iny ore? won,
HNY anderd fore thy my



 tensor([2.2635], grad_fn=<DivBackward0>) 

bre youe an peray and name too ath the hy wlave, what shere.

ENGrLIN':
Beant hre bene to have I and fores, chartrosh thard, :
mand nars lant frat bind, ang your mlod to is gof,all pey, to wine thave h



 tensor([2.1647], grad_fn=<DivBackward0>) 

bliy mile, the so mart it ouck and fot lice shawchim.
Whes rool, I wito my mantiofo pay oom dray be prerst likle, or ik is the fince, in the thy the bay on mut.

MINY RDo:
Inpoue the a friks cowior ful



 tensor([2.1938], grad_fn=<DivBackward0>) 

bse and wort
Whis acles as Lenreed us my 